In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [2]:
# Define the LEMCell
class LEMCell(nn.Module):
    def __init__(self, ninp, nhid, dt):
        super(LEMCell, self).__init__()
        self.ninp = ninp
        self.nhid = nhid
        self.dt = dt
        self.inp2hid = nn.Linear(ninp, 4 * nhid)
        self.hid2hid = nn.Linear(nhid, 3 * nhid)
        self.transform_z = nn.Linear(nhid, nhid)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.nhid)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, y, z):
        transformed_inp = self.inp2hid(x)
        transformed_hid = self.hid2hid(y)
        i_dt1, i_dt2, i_z, i_y = transformed_inp.chunk(4, 1)
        h_dt1, h_dt2, h_y = transformed_hid.chunk(3, 1)

        ms_dt_bar = self.dt * torch.sigmoid(i_dt1 + h_dt1)
        ms_dt = self.dt * torch.sigmoid(i_dt2 + h_dt2)

        z = (1. - ms_dt) * z + ms_dt * torch.tanh(i_y + h_y)
        y = (1. - ms_dt_bar) * y + ms_dt_bar * torch.tanh(self.transform_z(z) + i_z)

        return y, z

# Define the LEM model
class LEM(nn.Module):
    def __init__(self, ninp, nhid, nout, dt=1.):
        super(LEM, self).__init__()
        self.nhid = nhid
        self.cell = LEMCell(ninp, nhid, dt)
        self.classifier = nn.Linear(nhid, nout)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'classifier' in name and 'weight' in name:
                nn.init.kaiming_normal_(param.data)

    def forward(self, input):
        y = input.data.new(input.size(1), self.nhid).zero_()
        z = input.data.new(input.size(1), self.nhid).zero_()
        for x in input:
            y, z = self.cell(x, y, z)
        out = self.classifier(y)
        return out


### PINN data importing

In [3]:
# importing data

# Load the .mat file
mat_data = scipy.io.loadmat('burg.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u = mat_data['u1']



### Exact Solution data importing

In [4]:
# importing data

import torch
import torch.nn as nn
import numpy as np
import time
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('burgers_shock.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u_1 = mat_data['usol']


In [5]:
# Set random seed for reproducibility
#torch.manual_seed(42)

# Toy problem data
input_size = 256
hidden_size = 32
output_size = 256
sequence_length = 79
batch_size = 1
num_epochs = 20000

# Set random seed for reproducibility
#torch.manual_seed(42)
u[:, 0:100].shape

(256, 100)

In [6]:
input_data = u[:,0:79]
target_data = u[:,1:80]

test_data = u[:,79]
#test_target = u[:,80:100]

print("test data shape", test_data.shape)
#print("test target shape", test_target.shape)

print("input data shape",input_data.shape)
print("Target data shape",target_data.shape)

# Convert data to tensors
input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()

print("input tensor shape",input_tensor.shape)
print("Target tensor shape",target_tensor.shape)

test data shape (256,)
input data shape (256, 79)
Target data shape (256, 79)
input tensor shape torch.Size([1, 79, 256])
Target tensor shape torch.Size([1, 79, 256])


In [7]:
# Convert test data to tensors
test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
#test_target_tensor = torch.tensor(test_target.T).view(batch_size, 20, output_size).float()
target_tensor = torch.squeeze(target_tensor)

In [8]:
# Create LEM instance
lem = LEM(input_size, hidden_size, output_size, dt=0.9)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lem.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    output = lem(input_tensor)
    loss = criterion(output, target_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.16f}')

Epoch: 10/20000, Loss: 0.3878780901432037
Epoch: 20/20000, Loss: 0.3276734948158264
Epoch: 30/20000, Loss: 0.2693037092685699
Epoch: 40/20000, Loss: 0.2104976326227188
Epoch: 50/20000, Loss: 0.1560680568218231
Epoch: 60/20000, Loss: 0.1117625683546066
Epoch: 70/20000, Loss: 0.0791975110769272
Epoch: 80/20000, Loss: 0.0565306805074215
Epoch: 90/20000, Loss: 0.0415101125836372
Epoch: 100/20000, Loss: 0.0319545343518257
Epoch: 110/20000, Loss: 0.0261106137186289
Epoch: 120/20000, Loss: 0.0226338282227516
Epoch: 130/20000, Loss: 0.0205454155802727
Epoch: 140/20000, Loss: 0.0192088093608618
Epoch: 150/20000, Loss: 0.0182505883276463
Epoch: 160/20000, Loss: 0.0174629092216492
Epoch: 170/20000, Loss: 0.0167336184531450
Epoch: 180/20000, Loss: 0.0160037949681282
Epoch: 190/20000, Loss: 0.0152447950094938
Epoch: 200/20000, Loss: 0.0144468937069178
Epoch: 210/20000, Loss: 0.0136106647551060
Epoch: 220/20000, Loss: 0.0127425612881780
Epoch: 230/20000, Loss: 0.0118518397212029
Epoch: 240/20000, Lo

Epoch: 1900/20000, Loss: 0.0001453589356970
Epoch: 1910/20000, Loss: 0.0001447154936614
Epoch: 1920/20000, Loss: 0.0002242638147436
Epoch: 1930/20000, Loss: 0.0001478236663388
Epoch: 1940/20000, Loss: 0.0001448575058021
Epoch: 1950/20000, Loss: 0.0001375452557113
Epoch: 1960/20000, Loss: 0.0001355870626867
Epoch: 1970/20000, Loss: 0.0001341471215710
Epoch: 1980/20000, Loss: 0.0001324730983470
Epoch: 1990/20000, Loss: 0.0001310561201535
Epoch: 2000/20000, Loss: 0.0001310039078817
Epoch: 2010/20000, Loss: 0.0001883607474156
Epoch: 2020/20000, Loss: 0.0001531800226076
Epoch: 2030/20000, Loss: 0.0001282351877308
Epoch: 2040/20000, Loss: 0.0001260597055079
Epoch: 2050/20000, Loss: 0.0001246963074664
Epoch: 2060/20000, Loss: 0.0001230277557625
Epoch: 2070/20000, Loss: 0.0001219068144565
Epoch: 2080/20000, Loss: 0.0001208429093822
Epoch: 2090/20000, Loss: 0.0001199140169774
Epoch: 2100/20000, Loss: 0.0001190000257338
Epoch: 2110/20000, Loss: 0.0001181238621939
Epoch: 2120/20000, Loss: 0.00011

Epoch: 3880/20000, Loss: 0.0000699661250110
Epoch: 3890/20000, Loss: 0.0000849213756737
Epoch: 3900/20000, Loss: 0.0000648042623652
Epoch: 3910/20000, Loss: 0.0000667831118335
Epoch: 3920/20000, Loss: 0.0000626598994131
Epoch: 3930/20000, Loss: 0.0000628352427157
Epoch: 3940/20000, Loss: 0.0000626393448329
Epoch: 3950/20000, Loss: 0.0000703738332959
Epoch: 3960/20000, Loss: 0.0000636073527858
Epoch: 3970/20000, Loss: 0.0000626433175057
Epoch: 3980/20000, Loss: 0.0000616441320744
Epoch: 3990/20000, Loss: 0.0000608428090345
Epoch: 4000/20000, Loss: 0.0000604703855061
Epoch: 4010/20000, Loss: 0.0000601502470090
Epoch: 4020/20000, Loss: 0.0000602752734267
Epoch: 4030/20000, Loss: 0.0000694843474776
Epoch: 4040/20000, Loss: 0.0000861008520587
Epoch: 4050/20000, Loss: 0.0000650910224067
Epoch: 4060/20000, Loss: 0.0000598355763941
Epoch: 4070/20000, Loss: 0.0000591969437664
Epoch: 4080/20000, Loss: 0.0000585298294027
Epoch: 4090/20000, Loss: 0.0000579574480071
Epoch: 4100/20000, Loss: 0.00005

Epoch: 5800/20000, Loss: 0.0000212747963815
Epoch: 5810/20000, Loss: 0.0000191560775420
Epoch: 5820/20000, Loss: 0.0000185883818631
Epoch: 5830/20000, Loss: 0.0000188758695003
Epoch: 5840/20000, Loss: 0.0000295485187962
Epoch: 5850/20000, Loss: 0.0000423619312642
Epoch: 5860/20000, Loss: 0.0000210884772969
Epoch: 5870/20000, Loss: 0.0000191343806364
Epoch: 5880/20000, Loss: 0.0000184149412235
Epoch: 5890/20000, Loss: 0.0000176866342372
Epoch: 5900/20000, Loss: 0.0000173672742676
Epoch: 5910/20000, Loss: 0.0000174314245669
Epoch: 5920/20000, Loss: 0.0000221840036829
Epoch: 5930/20000, Loss: 0.0000556006889383
Epoch: 5940/20000, Loss: 0.0000315903962473
Epoch: 5950/20000, Loss: 0.0000198698999156
Epoch: 5960/20000, Loss: 0.0000181352879736
Epoch: 5970/20000, Loss: 0.0000174554807018
Epoch: 5980/20000, Loss: 0.0000183216579899
Epoch: 5990/20000, Loss: 0.0000413387788285
Epoch: 6000/20000, Loss: 0.0000266763145191
Epoch: 6010/20000, Loss: 0.0000195235679712
Epoch: 6020/20000, Loss: 0.00001

Epoch: 7670/20000, Loss: 0.0000136337957883
Epoch: 7680/20000, Loss: 0.0000120423619592
Epoch: 7690/20000, Loss: 0.0000086872414613
Epoch: 7700/20000, Loss: 0.0000091043430075
Epoch: 7710/20000, Loss: 0.0000183598531294
Epoch: 7720/20000, Loss: 0.0000221043264901
Epoch: 7730/20000, Loss: 0.0000130765556605
Epoch: 7740/20000, Loss: 0.0000113126288852
Epoch: 7750/20000, Loss: 0.0000087994212663
Epoch: 7760/20000, Loss: 0.0000089034620032
Epoch: 7770/20000, Loss: 0.0000115080683827
Epoch: 7780/20000, Loss: 0.0000376396565116
Epoch: 7790/20000, Loss: 0.0000141584914672
Epoch: 7800/20000, Loss: 0.0000107163741632
Epoch: 7810/20000, Loss: 0.0000098386099125
Epoch: 7820/20000, Loss: 0.0000091901511041
Epoch: 7830/20000, Loss: 0.0000131464585138
Epoch: 7840/20000, Loss: 0.0000212212053157
Epoch: 7850/20000, Loss: 0.0000109826623884
Epoch: 7860/20000, Loss: 0.0000097562169685
Epoch: 7870/20000, Loss: 0.0000093794233180
Epoch: 7880/20000, Loss: 0.0000114312751975
Epoch: 7890/20000, Loss: 0.00002

...

Epoch: 11400/20000, Loss: 0.0000041003813749
Epoch: 11410/20000, Loss: 0.0000034537249576
Epoch: 11420/20000, Loss: 0.0000047481116781
Epoch: 11430/20000, Loss: 0.0000135645877890
Epoch: 11440/20000, Loss: 0.0000091094825621
Epoch: 11450/20000, Loss: 0.0000158006514539
Epoch: 11460/20000, Loss: 0.0000113302285172
Epoch: 11470/20000, Loss: 0.0000056200042309
Epoch: 11480/20000, Loss: 0.0000034362774386
Epoch: 11490/20000, Loss: 0.0000031241222587
Epoch: 11500/20000, Loss: 0.0000031510935514
Epoch: 11510/20000, Loss: 0.0000050531662055
Epoch: 11520/20000, Loss: 0.0000283229837805
Epoch: 11530/20000, Loss: 0.0000120535651149
Epoch: 11540/20000, Loss: 0.0000048064234761
Epoch: 11550/20000, Loss: 0.0000040048257688
Epoch: 11560/20000, Loss: 0.0000036578644540
Epoch: 11570/20000, Loss: 0.0000041323646656
Epoch: 11580/20000, Loss: 0.0000123971367429
Epoch: 11590/20000, Loss: 0.0000062827962211
Epoch: 11600/20000, Loss: 0.0000088456490630
Epoch: 11610/20000, Loss: 0.0000110944602056
Epoch: 116

Epoch: 13300/20000, Loss: 0.0000033211861137
Epoch: 13310/20000, Loss: 0.0000049608347581
Epoch: 13320/20000, Loss: 0.0000141601258292
Epoch: 13330/20000, Loss: 0.0000130934167828
Epoch: 13340/20000, Loss: 0.0000128585706989
Epoch: 13350/20000, Loss: 0.0000048624160627
Epoch: 13360/20000, Loss: 0.0000033691515000
Epoch: 13370/20000, Loss: 0.0000021336541067
Epoch: 13380/20000, Loss: 0.0000019328674625
Epoch: 13390/20000, Loss: 0.0000021347116217
Epoch: 13400/20000, Loss: 0.0000066335646807
Epoch: 13410/20000, Loss: 0.0000117317467812
Epoch: 13420/20000, Loss: 0.0000034573977246
Epoch: 13430/20000, Loss: 0.0000026571733542
Epoch: 13440/20000, Loss: 0.0000036933929550
Epoch: 13450/20000, Loss: 0.0000289719664579
Epoch: 13460/20000, Loss: 0.0000231393150898
Epoch: 13470/20000, Loss: 0.0000066917668846
Epoch: 13480/20000, Loss: 0.0000036144633668
Epoch: 13490/20000, Loss: 0.0000022164711027
Epoch: 13500/20000, Loss: 0.0000017391040501
Epoch: 13510/20000, Loss: 0.0000016747820837
Epoch: 135

Epoch: 15210/20000, Loss: 0.0000015872783479
Epoch: 15220/20000, Loss: 0.0000021816711069
Epoch: 15230/20000, Loss: 0.0000195040993276
Epoch: 15240/20000, Loss: 0.0000162872547662
Epoch: 15250/20000, Loss: 0.0000049071513786
Epoch: 15260/20000, Loss: 0.0000034005313410
Epoch: 15270/20000, Loss: 0.0000023011273242
Epoch: 15280/20000, Loss: 0.0000016116962342
Epoch: 15290/20000, Loss: 0.0000015592166847
Epoch: 15300/20000, Loss: 0.0000016323192540
Epoch: 15310/20000, Loss: 0.0000047393282330
Epoch: 15320/20000, Loss: 0.0000216166063183
Epoch: 15330/20000, Loss: 0.0000132078675961
Epoch: 15340/20000, Loss: 0.0000044726989472
Epoch: 15350/20000, Loss: 0.0000023689021873
Epoch: 15360/20000, Loss: 0.0000018499770249
Epoch: 15370/20000, Loss: 0.0000017109073269
Epoch: 15380/20000, Loss: 0.0000018824538301
Epoch: 15390/20000, Loss: 0.0000053722073972
Epoch: 15400/20000, Loss: 0.0000178134105226
Epoch: 15410/20000, Loss: 0.0000049245463742
Epoch: 15420/20000, Loss: 0.0000027594144285
Epoch: 154

Epoch: 17110/20000, Loss: 0.0000018265252493
Epoch: 17120/20000, Loss: 0.0000015471379129
Epoch: 17130/20000, Loss: 0.0000023766599497
Epoch: 17140/20000, Loss: 0.0000192040297406
Epoch: 17150/20000, Loss: 0.0000062919120865
Epoch: 17160/20000, Loss: 0.0000047562721193
Epoch: 17170/20000, Loss: 0.0000022503645596
Epoch: 17180/20000, Loss: 0.0000014286800933
Epoch: 17190/20000, Loss: 0.0000013780528434
Epoch: 17200/20000, Loss: 0.0000013778849279
Epoch: 17210/20000, Loss: 0.0000017174022560
Epoch: 17220/20000, Loss: 0.0000122549618027
Epoch: 17230/20000, Loss: 0.0000139396079248
Epoch: 17240/20000, Loss: 0.0000056159819906
Epoch: 17250/20000, Loss: 0.0000022337592327
Epoch: 17260/20000, Loss: 0.0000019541373604
Epoch: 17270/20000, Loss: 0.0000014852919321
Epoch: 17280/20000, Loss: 0.0000013687165392
Epoch: 17290/20000, Loss: 0.0000016500197262
Epoch: 17300/20000, Loss: 0.0000101523164631
Epoch: 17310/20000, Loss: 0.0000057444485719
Epoch: 17320/20000, Loss: 0.0000041821308514
Epoch: 173

Epoch: 19100/20000, Loss: 0.0000015636430817
Epoch: 19110/20000, Loss: 0.0000118324051073
Epoch: 19120/20000, Loss: 0.0000079994315456
Epoch: 19130/20000, Loss: 0.0000064788632699
Epoch: 19140/20000, Loss: 0.0000019245057956
Epoch: 19150/20000, Loss: 0.0000017786588842
Epoch: 19160/20000, Loss: 0.0000013928739691
Epoch: 19170/20000, Loss: 0.0000020999418666
Epoch: 19180/20000, Loss: 0.0000137258821269
Epoch: 19190/20000, Loss: 0.0000034312965909
Epoch: 19200/20000, Loss: 0.0000026312939099
Epoch: 19210/20000, Loss: 0.0000025204301437
Epoch: 19220/20000, Loss: 0.0000020871634661
Epoch: 19230/20000, Loss: 0.0000020790600956
Epoch: 19240/20000, Loss: 0.0000080383342720
Epoch: 19250/20000, Loss: 0.0000044626244744
Epoch: 19260/20000, Loss: 0.0000039877563722
Epoch: 19270/20000, Loss: 0.0000047009025366
Epoch: 19280/20000, Loss: 0.0000050418543651
Epoch: 19290/20000, Loss: 0.0000033403925954
Epoch: 19300/20000, Loss: 0.0000019839737888
Epoch: 19310/20000, Loss: 0.0000016131285747
Epoch: 193

In [9]:
print(test_tensor.shape)
prediction_tensor = torch.zeros(1, 20, 256).float()
print(prediction_tensor.shape)

torch.Size([1, 1, 256])
torch.Size([1, 20, 256])


In [10]:
with torch.no_grad():
    prediction = lem(test_tensor)
    prediction = prediction.view(1, 1, 256).float()
    prediction_tensor[:, 0, :] = prediction
    for i in range(19):
        prediction = lem(prediction)
        prediction = prediction.view(1, 1, 256).float()
        prediction_tensor[:, i+1, :] = prediction
        

### Four different types of error

In [11]:
# Exact Solution

u_test = u_1.T
u_test_full = u_test[80:100, :]

In [12]:
prediction_tensor = torch.squeeze(prediction_tensor)


In [13]:
# Extrapolation

k1 = ( prediction_tensor - u_test_full)**2
u_test_full_tensor = torch.tensor(u_test_full**2)
u_test_full_tensor.shape

torch.Size([20, 256])

### L^2 norm error

In [14]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1)/ torch.mean(u_test_full_tensor)

print("Relative Error Test: ", relative_error_test.item(), "%")

Relative Error Test:  0.06202654228340688 %


In [15]:
2+

SyntaxError: invalid syntax (4209523232.py, line 1)

### Max absolute norm error

In [ ]:
R_abs = torch.max(torch.abs(prediction_tensor - u_test_full))

In [ ]:
print(R_abs)

### Explained variance score

In [ ]:
import torch

a = prediction_tensor
b = u_test_full
# Assuming 'a' is your predicted values (model's predictions) and 'b' is the true values (ground truth)
# Make sure 'a' and 'b' are PyTorch tensors
# a = torch.tensor(a)
b = torch.tensor(b)
# Calculate the mean of 'b'
mean_b = torch.mean(b)

# Calculate the Explained Variance Score
numerator = torch.var(b - a)  # Variance of the differences between 'b' and 'a'
denominator = torch.var(b)    # Variance of 'b'
evs = 1 - numerator / denominator

print("Explained Variance Score:", evs.item())

### Mean absolute error

In [ ]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(torch.abs(prediction_tensor - u_test_full))

print("Relative Error Test: ", relative_error_test, "%")

### Contour plot for PINN (80 percent) and (20 percentage lem prediction)

In [ ]:
print(prediction_tensor.shape)
prediction_tensor = torch.squeeze(prediction_tensor)
input_tensor = torch.squeeze(input_tensor)

conc_u = torch.squeeze(input_tensor)
concatenated_tensor = torch.cat((conc_u, prediction_tensor), dim=0)

x1 = np.linspace(-1, 1, 256)
t1 = np.linspace(0, 1, 99)

### Snapshot time plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed




final_time_output = prediction_tensor[3, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, 83].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.83}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.83_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.83_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed



final_time_output = prediction_tensor[-2, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, -2].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.98}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.98_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.98_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


### Contour plot where 80 percent for PINN solution and 20 percent for lem solution

### Exact contour

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = u_1.T

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

#plt.savefig('Contour_Exact.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_exact.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = concatenated_tensor.numpy()

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t1)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

# Add a dotted line at t = 0.8
plt.axvline(x=0.8, color='black', linestyle='dotted', linewidth=5)

#plt.savefig('Contour_LEM_20.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_LEM_20.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()
